## Load and prep the data for processing


In [70]:
import datetime
import pandas as pd
from pathlib import Path

# prep the environment
data_dir = Path.cwd() / "data"
uat_file_name = "ssrs report execution times.csv"
uat_file = data_dir / uat_file_name
data_file = data_dir / "uat_data.pkl"
uat_start = datetime.datetime(2019, 7, 22)


In [71]:
# load the UAT runtime data
raw_df = pd.read_csv(uat_file)


### Select Key Columns:
- All (for now)


In [72]:
# select key columns
uat_df = raw_df


### Format Columns:
1. DateTime columns = StartTime, EndTime


In [73]:
uat_df['StartTime'] = pd.to_datetime(uat_df['StartTime'], infer_datetime_format=True)
uat_df['StopTime'] = pd.to_datetime(uat_df['StopTime'], infer_datetime_format=True)


### Clean Invalid Rows:
1. Rows with NULL in the following columns:
  - User
  - ReportPath
1. Rows that were before the start of UAT

  

In [74]:
# clean invalid the rows
uat_df.dropna(subset=['User', 'ReportPath'], inplace=True)

# clean pre-UAT rows
uat_df = uat_df[uat_df['StartTime'] >= uat_start]


### Create Derived Fields:
1. DurationTotal - the sum of the 3 duration columns
1. DurationTotalSec - converting DurationTotal to seconds
1. DurationDataRetrievalSec - converting DurationDataRetrieval to seconds
1. DurationProcessingSec - converting DurationProcessing to seconds
1. DurationRenderingSec - converting DurationRendering to seconds
1. UserGroup - the broad group of users (directory)
1. ReportName - the name portion of the report
1. Agency - the Agency of the user who ran the report


In [75]:
# create derived fields
uat_df['DurationTotal'] = uat_df['DurationDataRetrieval'] + uat_df['DurationProcessing'] + uat_df['DurationRendering']
uat_df['DurationTotalSec'] = uat_df['DurationTotal'] / 1000
uat_df['DurationDataRetrievalSec'] = uat_df['DurationDataRetrieval'] / 1000
uat_df['DurationProcessingSec'] = uat_df['DurationProcessing'] / 1000
uat_df['DurationRenderingSec'] = uat_df['DurationRendering'] / 1000
# TODO: Change to ReportGroup
uat_df[['UserGroup', 'ReportName']] = uat_df['ReportPath'].str.extract('^/(?P<UserGroup>.*)/(?P<ReportName>.*$)')[['UserGroup', 'ReportName']]
uat_df['UserGroup'] = uat_df['UserGroup'].astype('category')
uat_df['ReportName'] = uat_df['ReportName'].astype('category')
# TODO: Add Agency


/Users/mcclure/PycharmProjects/dare_analytics/venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/mcclure/PycharmProjects/dare_analytics/venv/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/mcclure/PycharmProjects/dare_analytics/venv/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A va

### Save the processed DataFrame

In [76]:
# save the loaded data
uat_df.to_pickle(data_file)
print(f"Updated {data_file} on {datetime.datetime.now()}")

Updated /Users/mcclure/PycharmProjects/dare_analytics/data/uat_data.pkl on 2019-08-01 10:54:33.060901
